In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

pd.set_option('display.max_rows', 500)

2023-09-04 17:35:19.173017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# find file neame of training data
current_directory = os.listdir('./Data')
Xtest = [match for match in current_directory if 'Xtest' in match]
Xtrain = [match for match in current_directory if 'Xtrain' in match]
Ytest = [match for match in current_directory if 'Ytest' in match]
Ytrain = [match for match in current_directory if 'Ytrain' in match]

# read files into df's
df_Xtest = pd.read_csv('./Data/' + str(Xtest[0]), index_col=0)
df_Xtrain = pd.read_csv('./Data/' + str(Xtrain[0]), index_col=0)
df_Ytest = pd.read_csv('./Data/' + str(Ytest[0]), index_col=0)
df_Ytrain = pd.read_csv('./Data/' + str(Ytrain[0]), index_col=0)

In [3]:
df_Xtrain.info()
df_Ytrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79282 entries, 90402 to 85227
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SenecaGage                   79282 non-null  float64
 1   PointofRocksGage             79282 non-null  float64
 2   EdwardsGage                  79282 non-null  float64
 3   ShhepardstownGage            79282 non-null  float64
 4   HancockGage                  79282 non-null  float64
 5   SpringfieldGage              79282 non-null  float64
 6   SpringfieldGage_extrashift1  79282 non-null  float64
 7   SpringfieldGage_extrashift2  79282 non-null  float64
 8   SpringfieldGage_extrashift3  79282 non-null  float64
dtypes: float64(9)
memory usage: 6.0 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 79282 entries, 90402 to 85227
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   L

In [4]:
# lag variables
# positive lags shift obs down, so they push past, old obs down so they are next to current gage height
df['PointofRocksGage'] = df['PointofRocksGage'].shift(16) # 4hrs
df['EdwardsGage'] = df['EdwardsGage'].shift(12) # 3hrs
df['ShhepardstownGage'] = df['ShhepardstownGage'].shift(32) # 8hrs
df['HancockGage'] = df['HancockGage'].shift(60)   # 15hrs
df['SpringfieldGage'] = df['SpringfieldGage'].shift(68) # 17hrs

# to add more lagged fetures for 17hr prediction
df['SpringfieldGage_extrashift1'] = df['SpringfieldGage'].shift(1) # 15mins
df['SpringfieldGage_extrashift2'] = df['SpringfieldGage'].shift(2) # 30mins
df['SpringfieldGage_extrashift3'] = df['SpringfieldGage'].shift(3) # 45mins

df.drop(df.columns[[2,4,6,9,11,13]], axis=1, inplace=True)

In [5]:
df = df.dropna()

In [6]:
df.head(20)

,Date,LittleFallsGage,SenecaGage,PointofRocksGage,EdwardsGage,ShhepardstownGage,HancockGage,SpringfieldGage,SpringfieldGage_extrashift1,SpringfieldGage_extrashift2,SpringfieldGage_extrashift3
71,2022-04-13 07:00:00,4.99,2.35,5.26,7.88,5.85,7.45,3.96,3.97,3.97,3.98
72,2022-04-13 07:15:00,4.98,2.35,5.26,7.87,5.83,7.45,3.96,3.96,3.97,3.97
73,2022-04-13 07:30:00,4.97,2.35,5.25,7.87,5.82,7.44,3.96,3.96,3.96,3.97
74,2022-04-13 07:45:00,4.96,2.34,5.26,7.86,5.81,7.43,3.95,3.96,3.96,3.96
75,2022-04-13 08:00:00,4.97,2.34,5.25,7.85,5.81,7.43,3.95,3.95,3.96,3.96
76,2022-04-13 08:15:00,4.96,2.34,5.25,7.84,5.81,7.42,3.95,3.95,3.95,3.96
77,2022-04-13 08:30:00,4.96,2.34,5.24,7.84,5.79,7.41,3.94,3.95,3.95,3.95
78,2022-04-13 08:45:00,4.95,2.34,5.24,7.83,5.80,7.40,3.94,3.94,3.95,3.95
79,2022-04-13 09:00:00,4.95,2.34,5.24,7.82,5.78,7.40,3.93,3.94,3.94,3.95
80,2022-04-13 09:15:00,4.95,2.33,5.23,7.82,5.78,7.40,3.93,3.93,3.94,3.94


In [7]:
df_train = df.sample(frac=0.8,random_state=200)
df_test = df.drop(df_train.index)

In [8]:
Y_train = df_train['LittleFallsGage']
X_train = df_train.drop(columns=['Date', 'LittleFallsGage'])

In [9]:
Y_test = df_test['LittleFallsGage']
X_test = df_test.drop(columns=['Date', 'LittleFallsGage'])

Save the training and test into csv to build more

In [11]:
Y_train.to_csv('2022-04-12_throught_2023-04-12_cleaned_Ytrain.csv')
X_train.to_csv('2022-04-12_throught_2023-04-12_cleaned_Xtrain.csv')
Y_test.to_csv('2022-04-12_throught_2023-04-12_cleaned_Ytest.csv')
X_test.to_csv('2022-04-12_throught_2023-04-12_cleaned_Xtest.csv')


In [9]:
# need to break them out by seperate hourly predictions
#X_train_17hrs = X_train['SpringfieldGage']

In [10]:
X_train_17hrs = X_train[['SpringfieldGage','SpringfieldGage_extrashift1','SpringfieldGage_extrashift2','SpringfieldGage_extrashift3']].copy()
X_test_17hrs = X_test[['SpringfieldGage','SpringfieldGage_extrashift1','SpringfieldGage_extrashift2','SpringfieldGage_extrashift3']].copy()

## 17hr model: Train

In [11]:
#del(model_17hrs)
model_17hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation = 'tanh', kernel_regularizer='l2', input_shape = (4,)),  # input shape required
    tf.keras.layers.Dense(10, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(3, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(1, kernel_regularizer='l2')
])

model_17hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

2023-06-28 13:24:31.856221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# Train
model_17hrs.fit(X_train_17hrs , Y_train, epochs=150, verbose=0)

## 17hr model: Test

In [13]:
pred = model_17hrs.predict(X_test_17hrs)

620/620 [==============================] - 1s 799us/step


In [14]:
compare_df = pd.DataFrame(pred, columns=['Prediction'])
compare_df['TrueValue'] = Y_test.values
# used values bcuz...
# the index from the Y_test index is used to map it onto the new compare_df
# but the compare_df got its index form the prediction so it starts at 0 unlike Y_test
# so it only matches up rows correctly when the indexes match, which is few

In [15]:
compare_df.iloc[700:1000, :]

,Prediction,TrueValue
700,4.326269,4.43
701,4.323306,4.44
702,4.311455,4.41
703,4.305517,4.42
704,4.301051,4.42
705,4.295095,4.41
706,4.295095,4.41
707,4.295095,4.40
708,4.290631,4.40
709,4.289129,4.39


In [17]:
# Save the model
model_17hrs.save('Models/model_17hrs.h5')

## Test how to load and pred off model

In [2]:
new_model = tf.keras.models.load_model('Models/model_17hrs.h5')

2023-07-06 15:22:04.191334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# example to make one/two predicts
samples_to_predict = np.array(np.array([[5, 5, 5, 5], [4, 4, 4, 4]]))

new_model.predict(samples_to_predict)

1/1 [==============================] - 0s 37ms/step


array([[5.40041  ],
       [4.7992063]], dtype=float32)

## 15hr model: Train

In [19]:
model_15hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation = 'tanh', kernel_regularizer='l2', input_shape = (2,)),  # input shape required
    tf.keras.layers.Dense(10, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(3, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(1, kernel_regularizer='l2')
])

model_15hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

2023-08-31 12:49:20.126831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
model_15hrs.fit(df_Xtrain[['SpringfieldGage', 'HancockGage']] , df_Ytrain, epochs=150, verbose=0)

### 15hr model: Compare 

In [24]:
pred = model_15hrs.predict(df_Xtest[['SpringfieldGage', 'HancockGage']])
compare_df = pd.DataFrame(pred, columns=['Prediction'])
compare_df['TrueValue'] = df_Ytest['LittleFallsGage'].values
compare_df

620/620 [==============================] - 1s 814us/step


,Prediction,TrueValue
0,4.852535,4.96
1,4.852535,4.97
2,4.828933,4.93
3,4.821440,4.93
4,4.809734,4.91
...,...,...
19815,3.732164,3.52
19816,3.732164,3.51
19817,3.729778,3.51
19818,3.729778,3.52


### 15hr model: Save

In [26]:
model_15hrs.save('Models/model_15hrs.h5')

## 8hr model: Train

In [5]:
model_8hrs = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation = 'tanh', kernel_regularizer='l2', input_shape = (3,)),  # input shape required
    tf.keras.layers.Dense(10, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(3, activation = 'tanh', kernel_regularizer='l2'),
    tf.keras.layers.Dense(1, kernel_regularizer='l2')
])

model_8hrs.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error')

In [6]:
model_8hrs.fit(df_Xtrain[['SpringfieldGage', 'HancockGage', 'ShhepardstownGage']] , df_Ytrain, epochs=300, verbose=0)

## 8hr model: Compare

In [9]:
pred = model_8hrs.predict(df_Xtest[['SpringfieldGage', 'HancockGage', 'ShhepardstownGage']])
compare_df = pd.DataFrame(pred, columns=['Prediction'])
compare_df['TrueValue'] = df_Ytest['LittleFallsGage'].values
compare_df.head(200)

620/620 [==============================] - 1s 813us/step


,Prediction,TrueValue
0,4.721182,4.96
1,4.721182,4.97
2,4.691254,4.93
3,4.683000,4.93
4,4.670296,4.91
5,4.661698,4.90
6,4.607631,4.86
7,4.595057,4.82
8,4.593977,4.81
9,4.590386,4.81


## 8hr model: Save

In [10]:
model_8hrs.save('Models/model_8hrs.h5')